In [37]:
import datetime as dt
import ast
import json
from collections import Counter

import numpy as np
import pandas as pd
import string
import re

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestRegressor

In [38]:
df = pd.read_csv('new_cars.csv')
pd.set_option('display.max_columns', None)
df.head()

,link,listing_title,listing_mileage,primary_price,deal_gauge,exterior_color,interior_color,drivetrain,mpg,fuel_type,transmission,engine,vin,stock_number,vehicle_history,seller_name,price_history,Accidents or damage,1-owner vehicle,Personal use only,Open recall,Clean title,year,make,model,listed_date,price_change,price_change_percentage,Cylinders,Liters,Boosted,exterior_color_1,exterior_color_2,low mpg,high mpg
0,"""https://www.cars.com/vehicledetail/5ee3ae04-6...",2018 Hyundai Santa Fe Sport 2.4L,79497,17899,Good Deal $2 under,Nightfall Blue,Beige,Front-wheel Drive,21-27,Gasoline,6-Speed Automatic,2.4L I4 16V GDI DOHC,5XYZT3LB3JG542718,13669,"{'Accidents or damage': 'None reported', '1-ow...",ETHIO Motors,"[('9/21/21', 'Listed', '$20,299'), ('2/04/22',...",None reported,Yes,No,At least 1 open recall reported,NaN,2018,Hyundai,Santa Fe Sport 2.4L,2021-09-21,-2400.0,-11.823243,4.0,2.4,False,blue,NaN,21.0,27.0
1,"""https://www.cars.com/vehicledetail/c99e5035-7...",2017 RAM 1500 Express,80326,27900,This is a good deal.,Silver,Black,Four-wheel Drive,16-23,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,3C6RR7KT1HG533082,KBB2955,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('3/15/22', 'Listed', '$30,985'), ('6/01/22',...",None reported,No,Yes,NaN,NaN,2017,RAM,1500 Express,2022-03-15,-3085.0,-9.956431,8.0,5.7,False,silver,NaN,16.0,23.0
2,"""https://www.cars.com/vehicledetail/4e79fa3a-b...",2019 Nissan Kicks SR,92959,18950,This is a fair deal. Why?,Monarch Orange / Super Black,Charcoal,Front-wheel Drive,31-36,Gasoline,Automatic CVT,1.6L I4 16V MPFI DOHC,3N1CP5CU6KL510673,KBB3225,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('4/16/22', 'Listed', '$19,980'), ('6/01/22',...",None reported,No,Yes,NaN,NaN,2019,Nissan,Kicks SR,2022-04-16,-1030.0,-5.155155,4.0,1.6,False,orange,black,31.0,36.0
3,"""https://www.cars.com/vehicledetail/86a0d096-3...",2019 RAM 1500 Laramie,59176,39950,Great Deal $968 under,Gray,Black,Four-wheel Drive,19-24,Gasoline,8-Speed Automatic,5.7L V8 16V MPFI OHV,1C6SRFDT5KN869809,KBB3246,{'Accidents or damage': 'At least 1 accident o...,Custom Cars West,"[('6/01/22', 'Listed', '$40,523'), ('7/14/22',...",At least 1 accident or damage reported,No,No,At least 1 open recall reported,NaN,2019,RAM,1500 Laramie,2022-06-01,-573.0,-1.414012,8.0,5.7,False,gray,NaN,19.0,24.0
4,"""https://www.cars.com/vehicledetail/36efbef5-f...",2015 Jeep Wrangler Unlimited Rubicon,135385,26950,This is a good deal.,Blue,Black,Four-wheel Drive,16-21,Gasoline,-,3.6L V6 24V MPFI DOHC,1C4BJWFG2FL530006,KBB3145,"{'Accidents or damage': 'None reported', '1-ow...",Custom Cars West,"[('1/21/22', 'Listed', '$29,950'), ('3/22/22',...",None reported,No,Yes,At least 1 open recall reported,NaN,2015,Jeep,Wrangler Unlimited Rubicon,2022-01-21,-3000.0,-10.016694,6.0,3.6,False,blue,NaN,16.0,21.0


In [39]:
### Drop unncessary columns
columns_to_drop = ['link', 'stock_number', 'vin', 'deal_gauge', 'exterior_color', 'mpg', 'fuel_type',
                    'engine', 'price_history', 'vehicle_history', 'listing_title', 'interior_color',
                      'Open recall', 'Accidents or damage', 'exterior_color_2']
# add interior_color back once normalized
# add Open recall back once cleaned
# add Accidents or damage back once cleaned
df = df.drop(columns=columns_to_drop)

df.head()

,listing_mileage,primary_price,drivetrain,transmission,seller_name,1-owner vehicle,Personal use only,Clean title,year,make,model,listed_date,price_change,price_change_percentage,Cylinders,Liters,Boosted,exterior_color_1,low mpg,high mpg
0,79497,17899,Front-wheel Drive,6-Speed Automatic,ETHIO Motors,Yes,No,NaN,2018,Hyundai,Santa Fe Sport 2.4L,2021-09-21,-2400.0,-11.823243,4.0,2.4,False,blue,21.0,27.0
1,80326,27900,Four-wheel Drive,8-Speed Automatic,Custom Cars West,No,Yes,NaN,2017,RAM,1500 Express,2022-03-15,-3085.0,-9.956431,8.0,5.7,False,silver,16.0,23.0
2,92959,18950,Front-wheel Drive,Automatic CVT,Custom Cars West,No,Yes,NaN,2019,Nissan,Kicks SR,2022-04-16,-1030.0,-5.155155,4.0,1.6,False,orange,31.0,36.0
3,59176,39950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,No,No,NaN,2019,RAM,1500 Laramie,2022-06-01,-573.0,-1.414012,8.0,5.7,False,gray,19.0,24.0
4,135385,26950,Four-wheel Drive,-,Custom Cars West,No,Yes,NaN,2015,Jeep,Wrangler Unlimited Rubicon,2022-01-21,-3000.0,-10.016694,6.0,3.6,False,blue,16.0,21.0


In [40]:
### Convert data types
boolean_columns = ['1-owner vehicle', 'Personal use only', 'Clean title']

for column in boolean_columns:
    df[column] = df[column].map({'Yes': True, 'No': False}).astype(bool)

In [41]:
columns_with_missing_values = ['seller_name', 'Cylinders', 'Liters', 'exterior_color_1', 'low mpg', 'high mpg', 'listed_date','price_change','price_change_percentage']

# Drop rows with missing values in the specified columns
df = df.dropna(subset=columns_with_missing_values)

In [42]:
current_date = pd.Timestamp(dt.datetime.now().date())
df['listed_date'] = pd.to_datetime(df['listed_date'])
df['Days Avaliable'] = (current_date - df['listed_date']).dt.days
df.head()

,listing_mileage,primary_price,drivetrain,transmission,seller_name,1-owner vehicle,Personal use only,Clean title,year,make,model,listed_date,price_change,price_change_percentage,Cylinders,Liters,Boosted,exterior_color_1,low mpg,high mpg,Days Avaliable
0,79497,17899,Front-wheel Drive,6-Speed Automatic,ETHIO Motors,True,False,True,2018,Hyundai,Santa Fe Sport 2.4L,2021-09-21,-2400.0,-11.823243,4.0,2.4,False,blue,21.0,27.0,785
1,80326,27900,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,True,True,2017,RAM,1500 Express,2022-03-15,-3085.0,-9.956431,8.0,5.7,False,silver,16.0,23.0,610
2,92959,18950,Front-wheel Drive,Automatic CVT,Custom Cars West,False,True,True,2019,Nissan,Kicks SR,2022-04-16,-1030.0,-5.155155,4.0,1.6,False,orange,31.0,36.0,578
3,59176,39950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,2019,RAM,1500 Laramie,2022-06-01,-573.0,-1.414012,8.0,5.7,False,gray,19.0,24.0,532
4,135385,26950,Four-wheel Drive,-,Custom Cars West,False,True,True,2015,Jeep,Wrangler Unlimited Rubicon,2022-01-21,-3000.0,-10.016694,6.0,3.6,False,blue,16.0,21.0,663


In [29]:
df.head()

,listing_mileage,primary_price,drivetrain,transmission,seller_name,1-owner vehicle,Personal use only,Clean title,year,make,model,listed_date,price_change,price_change_percentage,Cylinders,Liters,Boosted,exterior_color_1,low mpg,high mpg
0,79497,17899,Front-wheel Drive,6-Speed Automatic,ETHIO Motors,True,False,True,2018,Hyundai,Santa Fe Sport 2.4L,2021-09-21,-2400.0,-11.823243,4.0,2.4,False,blue,21.0,27.0
1,80326,27900,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,True,True,2017,RAM,1500 Express,2022-03-15,-3085.0,-9.956431,8.0,5.7,False,silver,16.0,23.0
2,92959,18950,Front-wheel Drive,Automatic CVT,Custom Cars West,False,True,True,2019,Nissan,Kicks SR,2022-04-16,-1030.0,-5.155155,4.0,1.6,False,orange,31.0,36.0
3,59176,39950,Four-wheel Drive,8-Speed Automatic,Custom Cars West,False,False,True,2019,RAM,1500 Laramie,2022-06-01,-573.0,-1.414012,8.0,5.7,False,gray,19.0,24.0
4,135385,26950,Four-wheel Drive,-,Custom Cars West,False,True,True,2015,Jeep,Wrangler Unlimited Rubicon,2022-01-21,-3000.0,-10.016694,6.0,3.6,False,blue,16.0,21.0


## One Hot Encode Categorical 

In [30]:
categorical_columns = ['drivetrain', 'transmission', 'make', 'model', 'exterior_color_1', 'seller_name']
df = pd.get_dummies(df, columns=categorical_columns)


In [31]:
df.shape

(3860, 1690)

In [32]:
df.head()

listing_mileage  primary_price  1-owner vehicle  Personal use only  \
0            79497          17899             True              False   
1            80326          27900            False               True   
2            92959          18950            False               True   
3            59176          39950            False              False   
4           135385          26950            False               True   

   Clean title  year listed_date  price_change  price_change_percentage  \
0         True  2018  2021-09-21       -2400.0               -11.823243   
1         True  2017  2022-03-15       -3085.0                -9.956431   
2         True  2019  2022-04-16       -1030.0                -5.155155   
3         True  2019  2022-06-01        -573.0                -1.414012   
4         True  2015  2022-01-21       -3000.0               -10.016694   

   Cylinders  Liters  Boosted  low mpg  high mpg  drivetrain_-  \
0        4.0     2.4    False     21.0      27.0             0   
1        8.0     5.7    False     16.0      23.0             0   
2        4.0     1.6    False     31.0      36.0             0   
3        8.0     5.7    False     19.0      24.0             0   
4        6.0     3.6    False     16.0      21.0             0   

   drivetrain_All-wheel Drive  drivetrain_Four-wheel Drive  \
0                           0                            0   
1                           0                            1   
2                           0                            0   
3                           0                            1   
4                           0                            1   

   drivetrain_Front-wheel Drive  drivetrain_Rear-wheel Drive  \
0                             1                            0   
1                             0                            0   
2                             1                            0   
3                             0                            0   
4                             0                            0   

   drivetrain_Unknown  transmission_-  transmission_10-Speed Automatic  \
0                   0               0                                0   
1                   0               0                                0   
2                   0               0                                0   
3                   0               0                                0   
4                   0               1                                0   

   transmission_10-Speed Automatic with Overdrive  \
0                                               0   
1                                               0   
2                                               0   
3                                               0   
4                                               0   

   transmission_10-Speed Shiftable Automatic  transmission_4-Speed Automatic  \
0                                          0                               0   
1                                          0                               0   
2                                          0                               0   
3                                          0                               0   
4                                          0                               0   

   transmission_4-Speed Shiftable Automatic  transmission_5-Speed Automatic  \
0                                         0                               0   
1                                         0                               0   
2                                         0                               0   
3                                         0                               0   
4                                         0                               0   

   transmission_5-Speed Automatic with Overdrive  transmission_5-Speed Manual  \
0                                              0                            0   
1                                              0                            

## Build train test splits

In [33]:
# Create outcome and input variable lists
output = df['primary_price']
input_vars = df.drop(['primary_price'], axis=1)


in_train, in_test, out_train, out_test = train_test_split(input_vars, output, test_size=0.2, random_state=17)


## Feed into Model and predict

In [34]:
# Setup model
m = LinearRegression()
# Give input and output data
m.fit(in_train, out_train)


ValueError: could not convert string to float: '2023-10-21'